In [2]:
include("../../src/MetaDoE.jl")
using .MetaDoE: Experiments, ConstraintEnforcement, Constraints, PSO, Objectives, Designs, Models, HitAndRun, TensorOps
using Base.Iterators
using LinearAlgebra
using NPZ
using HiGHS
using Polyhedra
using Random

# 3D Mixture Example

In [3]:
# Specify number of trials and factors
K = 3
N = 12

# Define constraints
A = [
    0.0  -1.0   0.0;
    0.0   0.0   1.0;
    5.0   4.0   0.0;
  -20.0   5.0   0.0
]

b = [
   -1/10;
    3/5;
   39/10;
   -3
]

# Apply simplex constraints
A, b = Constraints.simplex(A, b)

# Define experiment
model = Models.full_cubic
experiment = Experiments.create(N, K-1, model)
experiment = Experiments.with_linear_constraints(experiment, A, b)

Main.MetaDoE.Experiments.Experiment(Dict("1" => 1, "2" => 2), Main.MetaDoE.ConstraintEnforcement.LinearConstraints([0.0 -1.0; -1.0 -1.0; … ; -0.0 -1.0; 1.0 1.0], [0.2333333333333333, 0.26666666666666666, 0.8999999999999999, 2.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333]), Main.MetaDoE.Models.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool, Bool}(1, 2, false, Any[], true, true), 12, 2)

In [43]:
# Create optimization context
objective = Objectives.D
params = PSO.create_hyperparams(500)
context = PSO.create_context(experiment, objective; hyperparams=params)

# Optimize
runner_state, history = PSO.optimize(context)

# Extract optimizer
optimizer, optimum = PSO.get_optimizer(runner_state)

# Save samples
sampler = HitAndRun.hit_and_run(experiment.constraints.A, experiment.constraints.b)
samples = sampler(10_000)
npzwrite("../data/coetzer_2d_samples.npy", samples)

# Save optimizer
npzwrite("../data/coetzer_2d_12.npy", optimizer)

# Save vertices
Constraints.save_vertices(experiment; location = "../data/coetzer_verts_2d.npy")

Iteration: 0 Best score: 9.269403059847914
Iteration: 1 Best score: 8.845832423534414
Iteration: 2 Best score: 8.332203526089913
Iteration: 3 Best score: 8.332203526089913
Iteration: 4 Best score: 8.332203526089913
Iteration: 5 Best score: 8.332203526089913
Iteration: 6 Best score: 8.332203526089913
Iteration: 7 Best score: 8.332203526089913
Iteration: 8 Best score: 8.332203526089913
Iteration: 9 Best score: 8.332203526089913
Iteration: 10 Best score: 8.332203526089913
Iteration: 11 Best score: 8.332203526089913
Iteration: 12 Best score: 8.332203526089913
Iteration: 13 Best score: 8.332203526089913
Iteration: 14 Best score: 8.332203526089913
Iteration: 15 Best score: 8.332203526089913
Iteration: 16 Best score: 8.332203526089913
Iteration: 17 Best score: 8.332203526089913
Iteration: 18 Best score: 8.332203526089913
Iteration: 19 Best score: 8.332203526089913
Iteration: 20 Best score: 8.332203526089913
Iteration: 21 Best score: 8.332203526089913
Iteration: 22 Best score: 8.33220352608991

In [45]:
verts = Constraints.get_vertices(experiment)
simp_verts = Constraints.deparameterize_simplex(verts)

4×3 Matrix{Float64}:
 0.3  0.6  0.1
 0.2  0.2  0.6
 0.7  0.1  0.2
 0.3  0.1  0.6

In [5]:
# Save samples
sampler = HitAndRun.hit_and_run(experiment.constraints.A, experiment.constraints.b)
samples = sampler(10_000)
npzwrite("../data/coetzer_2d_samples.npy", samples)

# Save optimizer
npzwrite("../data/coetzer_2d_12.npy", optimizer)

# Save vertices
Constraints.save_vertices(experiment; location = "../data/coetzer_verts_2d.npy")

# 4D Mixture Example

In [3]:
N = 12
K = 4

A = [
    -1.0  0.0  0.0  0.0;
     1.0  0.0  0.0  0.0;
     0.0 -1.0  0.0  0.0;
     0.0  1.0  0.0  0.0;
     0.0  0.0 -1.0  0.0;
     0.0  0.0  1.0  0.0;
     0.0  0.0  0.0 -1.0;
     0.0  0.0  0.0  1.0
]

b = [
    -0.2;
     0.65;
    -0.1;
     0.55;
    -0.1;
     0.2;
    -0.15;
     0.35
]

A, b = Constraints.simplex(A, b)

model = Models.scheffe(2)
experiment = Experiments.create(N, K-1, model)
experiment = Experiments.with_linear_constraints(experiment, A, b)

Main.MetaDoE.Experiments.Experiment(Dict("1" => 1, "2" => 2, "3" => 3), Main.MetaDoE.ConstraintEnforcement.LinearConstraints([-1.0 0.0 0.0; 1.0 0.0 0.0; … ; -0.0 -0.0 -1.0; 1.0 1.0 1.0], [0.04999999999999999, 0.4, 0.15, 0.30000000000000004, 0.15, -0.04999999999999999, 0.1, 0.09999999999999998, 0.25, 0.25, 0.25, 0.25]), Main.MetaDoE.Models.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool, Bool}(1, 1, false, Any[], false, true), 12, 3)

In [4]:
context = PSO.create_context(experiment, Objectives.D)
runner_state, history = PSO.optimize(context)
optimizer, optimum = PSO.get_optimizer(runner_state)

# Save samples
sampler = HitAndRun.hit_and_run(experiment.constraints.A, experiment.constraints.b)
samples = sampler(10000)
npzwrite("../data/coetzer_3d_samples.npy", samples)

# Save optimizer
npzwrite("../data/coetzer_3d_12.npy", optimizer)

# Save vertices
Constraints.save_vertices(experiment; location = "../data/coetzer_verts_3d.npy")

Iteration: 0 Best score: 15.618488287102522
Iteration: 1 Best score: 15.33923865192352
Iteration: 2 Best score: 15.33923865192352
Iteration: 3 Best score: 15.33923865192352
Iteration: 4 Best score: 15.33923865192352
Iteration: 5 Best score: 15.33923865192352
Iteration: 6 Best score: 15.33923865192352
Iteration: 7 Best score: 15.33923865192352
Iteration: 8 Best score: 15.33923865192352
Iteration: 9 Best score: 15.33923865192352
Iteration: 10 Best score: 15.33923865192352
Iteration: 11 Best score: 15.33923865192352
Iteration: 12 Best score: 15.33923865192352
Iteration: 13 Best score: 15.33923865192352
Iteration: 14 Best score: 15.33923865192352
Iteration: 15 Best score: 15.33923865192352
Iteration: 16 Best score: 15.33923865192352
Iteration: 17 Best score: 15.33923865192352
Iteration: 18 Best score: 15.33923865192352
Iteration: 19 Best score: 15.33923865192352
Iteration: 20 Best score: 15.33923865192352
Iteration: 21 Best score: 15.33923865192352
Iteration: 22 Best score: 15.339238651923